# Wired CSV

The challenge description states:
> We have a photo and a CSV file. NOTE: The flag does not follow the CTF{...} format, but is clearly marked as the flag. Please add the CTF{...} around the flag manually when submitting.

We download a zip file containing a photo, and another 7z archive containing a big data.csv file. The CSV file contains a dump of a logic analyzer. We first take a look at the picture:

![wired](img/wires.jpg)

In the picture we see a logic analyzer connected to a chip. Googling the markings (C012294B) learns us that this a [POKEY](https://en.wikipedia.org/wiki/POKEY) (Pot Keyboard Integrated Circuit) chip, and we're looking at the PCB of an Atari. Wikipedia gives us the pinout:

![pinout](img/pokey_pinout.png)

If we compare the pinout to the photo we can see the logic analyzer is connected to the pins driving the keyboard (K0-K5, KR1-KR2). So the rest of the challenge probably consists of using the logic analyzer output to reconstruct some text typed on the keyboard. Now we can look at the logic analyzer output.

In [1]:
import pandas
inp = pandas.read_csv('data.csv')
inp.head()

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Time [s],Wire6-Analog,Wire7-Analog,Time [s].1,Wire0-Digital,Time [s].2,Wire1-Digital,Time [s].3,Wire2-Digital,Time [s].4,Wire3-Digital,Time [s].5,Wire4-Digital,Time [s].6,Wire5-Digital,Time [s].7,Wire6-Digital,Time [s].8,Wire7-Digital
0,0.000000,4.768121,4.773900,0,0,0,0,0,0,0,1,0.000000000000000,0,0.000000000000000,0,0.000000000000000,1,0.000000000000000,1.0
1,0.000008,4.768121,4.773900,9.9e-07,1,6.556e-05,1,0.00019438,1,0.00045175,0,0.000452070000000,1,0.001480790000000,1,1.468471380000000,0,2.503182740000000,0.0
2,0.000016,4.773141,4.778934,6.523e-05,0,0.00019407,0,0.00045145,0,0.00096599,1,0.001480440000000,0,0.003537600000000,0,1.468535670000000,1,2.503689840000000,1.0
3,0.000024,4.773141,4.773900,0.00012954,1,0.00032266,1,0.0007086,1,0.00148017,0,0.002508920000000,1,0.005594510000000,1,1.472585100000000,0,2.507288860000000,0.0
4,0.000032,4.773141,4.773900,0.00019378,0,0.00045118,0,0.00096566,0,0.00199442,1,0.003537300000000,0,0.007651320000000,0,1.472649390000000,1,2.507799430000000,1.0


The logic analyzer log contains channels 0-8 in digital, and channel 6 and 7 are also stored as analog values (which we don't need). The digital values are only stored when their value changes, accompanied by a timestamp.

Some more googling gives us the hardware connected to the keyboard pins:
![pinout](img/matrix.gif)

K0-K5 go into a 4051 8-to-1 multiplexer/demultiplexer chips. This means that the OUT pin of the 4051 is connected to one its 8 input according to the three bit control signal. The Atari contains two of these 4051s connected to K0,K1,K2 and K3,K4,K5 respectively. The POKEY chip will loop through all possible values of K0-K5, and by doing this it connects each key to KR1 indivually. (KR2 is used for CTRL, SHIFT and BRK which turned out to not be neccesary for this challenge).

To read the keyboard we have to check when KR1 goes low (the input is active low) and then read the value of K0-K5 to get the position in the keyboard matrix of the key pressed. 

By looking up the wire colors of the logic anlyzer we can give the signals some proper names.

In [2]:
# Values
K0 = 'Wire0-Digital'
K1 = 'Wire1-Digital' 
K2 = 'Wire2-Digital' 
K3 = 'Wire3-Digital' 
K4 = 'Wire4-Digital' 
K5 = 'Wire5-Digital' 
KR1 = 'Wire6-Digital' 
KR2 = 'Wire7-Digital' 

# Timestamps
T_K0 = ' Time [s].1'
T_K1 = ' Time [s].2'
T_K2 = ' Time [s].3'
T_K3 = ' Time [s].4'
T_K4 = ' Time [s].5'
T_K5 = ' Time [s].6'
T_KR1 = ' Time [s].7'
T_KR2 = ' Time [s].8'

In [3]:
import numpy as np

def convert(t1, v1):
    """ Convert timestamps to floats, and binary values to ints """
    t1 = inp[t1].values
    v1 = inp[v1].values
    t1 = list(map(float, [a for a in t1 if a != ' ']))
    v1 = list(map(int, v1[:len(t1)]))
    return np.array(t1), np.array(v1)

tk0, k0 = convert(T_K0, K0)
tk1, k1 = convert(T_K1, K1)
tk2, k2 = convert(T_K2, K2)
tk3, k3 = convert(T_K3, K3)
tk4, k4 = convert(T_K4, K4)
tk5, k5 = convert(T_K5, K5)
tkr1, kr1 = convert(T_KR1, KR1)
tkr2, kr2 = convert(T_KR2, KR2)

In [4]:
# We also have to make a dictory of which key corresponds to which keyboard code
# https://atariage.com/forums/topic/188172-pokey-keyboard-codes/
KEYMAP = "KEY_A  = 63\nKEY_S  = 62\nKEY_G  = 61\nKEY_Cap  = 60\nKEY_D  = 58\nKEY_H  = 57\nKEY_F  = 56\nKEY_>  = 55\nKEY_<  = 54\nKEY_8  = 53\nKEY_BSp  = 52\nKEY_7  = 51\nKEY_0  = 50\nKEY_9  = 48\nKEY_Q  = 47\nKEY_W  = 46\nKEY_T  = 45\nKEY_Tab  = 44\nKEY_Y  = 43\nKEY_E  = 42\nKEY_R  = 40\nKEY_Inv  = 39\nKEY_/  = 38\nKEY_M  = 37\nKEY_N  = 35\nKEY_.  = 34\nKEY__  = 33\nKEY_,  = 32\nKEY_1  = 31\nKEY_2  = 30\nKEY_5  = 29\nKEY_Esc  = 28\nKEY_6  = 27\nKEY_3  = 26\nKEY_4  = 24\nKEY_Z  = 23\nKEY_X  = 22\nKEY_B  = 21\nKEY_F4  = 20\nKEY_F3  = 19\nKEY_C  = 18\nKEY_Hlp  = 17\nKEY_V  = 16\nKEY_=  = 15\nKEY_-  = 14\nKEY_I  = 13\nKEY_\\n  = 12\nKEY_U  = 11\nKEY_P  = 10\nKEY_O  = 8\nKEY_Aster  = 7\nKEY_plus  = 6\nKEY_K  = 5\nKEY_F2  = 4\nKEY_F1  = 3\nKEY_;  = 2\nKEY_J  = 1\nKEY_L  = 0"

keys = {}
for k in KEYMAP.split('\n'):
    k = k.split('  = ')
    keys[int(k[1])] = k[0].replace('KEY_', '')

Now it's time to read the actual keyboard codes. We are interested when KR0 line goes from a 1 to a 0. Luckily only changes in value are stored in the CSV. So we take all odd entries in the list of timestamps of KR1.

Then we have to calculate the keycodes at that timestamp from K0-K5. This means we want to find the last change that happened before that timestamp for each of those signals.

In [5]:
from bitstring import BitArray

def get_at_time(t, v, t0):
    """ Get value of signal v at t0. 
    Find the index with the last change before this timestamp and return the value.
    """
    ii = np.argmax(t > t0)
    return 1 - v[ii-1] # The signals are active low, so invert

def get_keycode(t):
    v0 = get_at_time(tk0, k0, t)
    v1 = get_at_time(tk1, k1, t)
    v2 = get_at_time(tk2, k2, t)
    v3 = get_at_time(tk3, k3, t)
    v4 = get_at_time(tk4, k4, t)
    v5 = get_at_time(tk5, k5, t)
    
    return BitArray([v5, v4, v3, v2, v1, v0]).uint

The last step of finding the key is combing the functions we wrote above, and adding some debounce logic. When you press a button on a physical keyboard the electrical signal won't simply change from a 0 to 1. Due to the physical nature of the switch the value will change a few times before settling on the final value. Therefore we enforce a minimal time between consecutive key presses.

In [6]:
last_key_t = 0
s = ""

for i in range(1, len(kr1), 2):
    dt = tkr1[i] - last_key_t
    if dt > 0.05: # Debounce logic
        last_key_t = tkr1[i]
        
        kk = get_keycode(tkr1[i])
        s += keys[kk]
        
print(s)

FLAG;_8-BIT-HARDWARE-KEYLOGGER\n


**This gives us the flag CTF{8-BIT-HARDWARE-KEYLOGGER}**